In [1]:
DEVELOPMENT_MODE = False
try:
    import google.colab

    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/jbloomAus/SAELens

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_1734/740336833.py:16: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_1734/740336833.py:17: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
import torch
if torch.cuda.is_available():
    device = "cuda"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"
torch.set_grad_enabled(False)

In [3]:
from sae_lens import HookedSAETransformer

model: HookedSAETransformer = HookedSAETransformer.from_pretrained("gpt2-small").to(
    device
)

/mnt/d/code/sae/SAE_Geometry/.venv/lib/python3.12/site-packages/einops/einops.py:827: SyntaxWarning: invalid escape sequence '\s'
  """
/mnt/d/code/sae/SAE_Geometry/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


In [44]:
import tqdm
from sae_lens import SAE

hook_name_to_sae = {}
for layer in tqdm.tqdm(range(12)):
    sae, cfg_dict, _ = SAE.from_pretrained(
        "gpt2-small-res-jb",
        f"blocks.{layer}.hook_resid_pre",
        device=device,
    )
    hook_name_to_sae[sae.cfg.hook_name] = sae

print(hook_name_to_sae.keys())

100%|███████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.91it/s]

dict_keys(['blocks.0.hook_resid_pre', 'blocks.1.hook_resid_pre', 'blocks.2.hook_resid_pre', 'blocks.3.hook_resid_pre', 'blocks.4.hook_resid_pre', 'blocks.5.hook_resid_pre', 'blocks.6.hook_resid_pre', 'blocks.7.hook_resid_pre', 'blocks.8.hook_resid_pre', 'blocks.9.hook_resid_pre', 'blocks.10.hook_resid_pre', 'blocks.11.hook_resid_pre'])


In [45]:
import transformer_lens.utils as utils

layer = 7
layer_name = f"blocks.{layer}.hook_resid_pre"
sae = hook_name_to_sae[layer_name]
print(sae.cfg)

SAEConfig(architecture='standard', d_in=768, d_sae=24576, activation_fn_str='relu', apply_b_dec_to_input=True, finetuning_scaling_factor=False, context_size=128, model_name='gpt2-small', hook_name='blocks.7.hook_resid_pre', hook_layer=7, hook_head_index=None, prepend_bos=True, dataset_path='Skylion007/openwebtext', dataset_trust_remote_code=True, normalize_activations='none', dtype='torch.float32', device='cuda', sae_lens_training_version=None, activation_fn_kwargs={}, neuronpedia_id='gpt2-small/7-res-jb', model_from_pretrained_kwargs={'center_writing_weights': True}, seqpos_slice=(None,))


In [46]:
print(cfg_dict)

{'model_name': 'gpt2-small', 'hook_point': 'blocks.11.hook_resid_pre', 'hook_point_layer': 11, 'hook_point_head_index': None, 'dataset_path': 'Skylion007/openwebtext', 'is_dataset_tokenized': False, 'context_size': 128, 'use_cached_activations': False, 'cached_activations_path': 'activations/Skylion007_openwebtext/gpt2-small/blocks.11.hook_resid_pre', 'd_in': 768, 'n_batches_in_buffer': 128, 'total_training_tokens': 300000000, 'store_batch_size': 32, 'device': 'cuda', 'seed': 42, 'dtype': 'torch.float32', 'b_dec_init_method': 'geometric_median', 'expansion_factor': 32, 'from_pretrained_path': None, 'l1_coefficient': 8e-05, 'lr': 0.0004, 'lr_scheduler_name': None, 'lr_warm_up_steps': 5000, 'train_batch_size': 4096, 'use_ghost_grads': False, 'feature_sampling_window': 1000, 'feature_sampling_method': None, 'resample_batches': 1028, 'feature_reinit_scale': 0.2, 'dead_feature_window': 5000, 'dead_feature_estimation_method': 'no_fire', 'dead_feature_threshold': 1e-08, 'log_to_wandb': True, 

In [47]:
prompt_format = [
    "When John and Mary went to the shops,{} gave the bag to",
    "When Tom and James went to the park,{} gave the ball to",
    "When Dan and Sid went to the shops,{} gave an apple to",
    "After Martin and Amy went to the park,{} gave a drink to",
]
names = [
    (
        " John",
        " Mary",
    ),
    (" Tom", " James"),
    (" Dan", " Sid"),
    (" Martin", " Amy"),
]
# List of prompts
prompts = []
# List of answers, in the format (correct, incorrect)
answers = []
# List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
answer_tokens = []
for i in range(len(prompt_format)):
    for j in range(2):
        answers.append((names[i][j], names[i][1 - j]))
        answer_tokens.append(
            (
                model.to_single_token(answers[-1][0]),
                model.to_single_token(answers[-1][1]),
            )
        )
        # Insert the *incorrect* answer to the prompt, making the correct answer the indirect object.
        prompts.append(prompt_format[i].format(answers[-1][1]))
answer_tokens = torch.tensor(answer_tokens).to(device)
print(prompts)
print(answers)

['When John and Mary went to the shops, Mary gave the bag to', 'When John and Mary went to the shops, John gave the bag to', 'When Tom and James went to the park, James gave the ball to', 'When Tom and James went to the park, Tom gave the ball to', 'When Dan and Sid went to the shops, Sid gave an apple to', 'When Dan and Sid went to the shops, Dan gave an apple to', 'After Martin and Amy went to the park, Amy gave a drink to', 'After Martin and Amy went to the park, Martin gave a drink to']
[(' John', ' Mary'), (' Mary', ' John'), (' Tom', ' James'), (' James', ' Tom'), (' Dan', ' Sid'), (' Sid', ' Dan'), (' Martin', ' Amy'), (' Amy', ' Martin')]


In [48]:
_, cache = model.run_with_cache_with_saes(tokens, saes=[sae])

In [49]:
print(cache)

ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'blocks.2.hook_re

In [51]:
# [bz, sz, saz] (batch_size, sequence_size, sae_activation_size)
cache['blocks.7.hook_resid_pre.hook_sae_acts_post'].shape

torch.Size([8, 15, 24576])

In [53]:
sae_acts = cache['blocks.7.hook_resid_pre.hook_sae_acts_post']
live_feature = sae_acts > 0
live_feature_union = live_feature.any(dim=0)

In [55]:
live_feature_union.shape

torch.Size([15, 24576])

In [56]:
torch.max(live_feature_union)

tensor(True, device='cuda:0')

In [66]:
# TODO: speed up

def cosine_similarity_by_column(matrix):
    # Normalize each column vector
    matrix = matrix.float()
    cos_sim = torch.zeros(matrix.shape[1], matrix.shape[1])
    for i in tqdm.tqdm(range(matrix.shape[1])):
        cos_sim[:, i] = torch.mm(matrix.T, matrix[:, i].unsqueeze(1)).reshape(-1)
    
    return cos_sim

# Example usage
matrix = torch.randn(3, 5)  # Example matrix with 5 rows and 3 columns
cos_sim_matrix = cosine_similarity_by_column(matrix)
print(cos_sim_matrix)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10979.85it/s]

tensor([[ 0.1097, -0.3123, -0.1200,  0.1681,  0.1030],
        [-0.3123,  3.0602, -0.4328,  0.5485,  0.3888],
        [-0.1200, -0.4328,  1.2228, -0.5980, -0.7856],
        [ 0.1681,  0.5485, -0.5980,  0.7467,  0.5058],
        [ 0.1030,  0.3888, -0.7856,  0.5058,  0.5374]])


In [68]:
sim = cosine_similarity_by_column(live_feature_union)

100%|███████████████████████████████████████████████████████████████████████████████████| 24576/24576 [00:10<00:00, 2256.06it/s]


In [69]:
torch.max(sim)

tensor(13.)

In [70]:
sim.shape

torch.Size([24576, 24576])

In [72]:
torch.nonzero(sim).shape

torch.Size([146527, 2])

In [73]:
from datasets import load_dataset

# Load the Skylion007/openwebtext dataset
dataset = load_dataset('Skylion007/openwebtext')

# Print the first example from the dataset
print(dataset['train'][0])

/mnt/d/code/sae/SAE_Geometry/.venv/lib/python3.12/site-packages/datasets/load.py:1454: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 100%|███████████████████████████████████████████████| 8013769/8013769 [11:13<00:00, 11894.30 examples/s]


{'text': 'Port-au-Prince, Haiti (CNN) -- Earthquake victims, writhing in pain and grasping at life, watched doctors and nurses walk away from a field hospital Friday night after a Belgian medical team evacuated the area, saying it was concerned about security.\n\nThe decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.\n\nCNN initially reported, based on conversations with some of the doctors, that the United Nations ordered the Belgian First Aid and Support Team to evacuate. However, Belgian Chief Coordinator Geert Gijs, a doctor who was at the hospital with 60 Belgian medical personnel, said it was his decision to pull the team out for the night. Gijs said he requested U.N. security personnel to staff the hospital overnight, but was told that peacekeepers would only be able to evacuate the team.\n\nHe said it was a "tough decision" but that he accepted the U.N. offer to evacuate after a Canadian medical t

In [74]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8013769
    })
})

In [78]:
dataset['train'][222]

{'text': "This is the bizarre moment an Asian woman praised Pauline Hanson by comparing her to both Donald Trump and Adolf Hitler.\n\nThe woman's gushing praise of the controversial One Nation leader was captured in an interview with ABC's The Link presenter Stan Grant.\n\nHe spoke to residents of New South Wales in a segment that aired on Friday night asking them what they think of Ms Hanson.\n\nWhile one man dismissed her as 'too anti-everything,' an Asian woman – who was not named – was filmed in a shopping centre as she heaped praise on Senator Hanson.\n\nThis woman heaped praise Pauline Hanson in an interview with The Link - comparing her to Donald Trump and Adolf Hitler\n\n'Pauline Hanson is straightforward,' she said.\n\n'She's sincere, she's not a hypocrite. If she doesn't like you, I don't like you.'\n\nThe woman continued to compliment Ms Hanson by comparing her qualities with two other nationalistic politicians she admires - the current President of the United States and the

In [87]:
example = dataset['train'][222]
print(example['text'])
tokens = model.to_tokens(example['text'], prepend_bos=True)

This is the bizarre moment an Asian woman praised Pauline Hanson by comparing her to both Donald Trump and Adolf Hitler.

The woman's gushing praise of the controversial One Nation leader was captured in an interview with ABC's The Link presenter Stan Grant.

He spoke to residents of New South Wales in a segment that aired on Friday night asking them what they think of Ms Hanson.

While one man dismissed her as 'too anti-everything,' an Asian woman – who was not named – was filmed in a shopping centre as she heaped praise on Senator Hanson.

This woman heaped praise Pauline Hanson in an interview with The Link - comparing her to Donald Trump and Adolf Hitler

'Pauline Hanson is straightforward,' she said.

'She's sincere, she's not a hypocrite. If she doesn't like you, I don't like you.'

The woman continued to compliment Ms Hanson by comparing her qualities with two other nationalistic politicians she admires - the current President of the United States and the leader of Nazi Germany.

In [89]:
tokens.shape

torch.Size([1, 631])

In [91]:
_, cache = model.run_with_cache_with_saes(tokens, saes=sae)

In [92]:
sae_acts = cache['blocks.7.hook_resid_pre.hook_sae_acts_post']
live_feature = sae_acts > 0
live_feature_union = live_feature.any(dim=0)
live_feature_union.shape

torch.Size([631, 24576])

In [93]:
sim = cosine_similarity_by_column(live_feature_union)

100%|███████████████████████████████████████████████████████████████████████████████████| 24576/24576 [00:12<00:00, 2013.18it/s]


In [94]:
torch.max(sim)

tensor(495.)

In [96]:
seq_sim = torch.sum(sim, 1)

In [97]:
seq_sim.shape

torch.Size([24576])

In [112]:
live_feature_union.sum(0).shape

torch.Size([24576])

In [109]:
tmp = live_feature_union.sum(0)
torch.max(tmp)

tensor(495, device='cuda:0')

In [ ]:
for idx in tqdm.tqdm(range(len(dataset['train']))):
    #for idx in tqdm.tqdm(range(10)):
    example = dataset['train'][idx]
    tokens = model.to_tokens(example['text'], prepend_bos=True)
    _, cache = model.run_with_cache_with_saes(tokens, saes=sae)
    sae_acts = cache['blocks.7.hook_resid_pre.hook_sae_acts_post']

In [147]:
import numpy as np
with open("activation.npy", "wb") as f:
    for idx in tqdm.tqdm(range(len(dataset['train']))):
    #for idx in tqdm.tqdm(range(10)):
        example = dataset['train'][idx]
        tokens = model.to_tokens(example['text'], prepend_bos=True)
        _, cache = model.run_with_cache_with_saes(tokens, saes=sae)
        sae_acts = cache['blocks.7.hook_resid_pre.hook_sae_acts_post']
        # live_feature = sae_acts > 0
        # live_feature_union = live_feature.any(dim=0)
        seq_sim = sae_acts.sum(0)
        np.save(f, seq_sim.cpu().detach().numpy())

  0%|                                                                                  | 31/8013769 [00:07<540:22:43,  4.12it/s]


KeyboardInterrupt: 

In [144]:
with open('activation.npy', 'rb') as f:
    activation1 = np.load(f)
    activation2 = np.load(f)
print(activation2)
print(activation1)


[  0  11   0 ...  18 164   4]
[  0  14   0 ...  53 157  20]


In [132]:
with open('test.npy', 'wb') as f:
    np.save(f, np.array([1, 2]))
    np.save(f, np.array([1, 3]))

In [143]:
with open('test.npy', 'rb') as f:
    print(np.load(f))
    print(np.load(f))
    print(np.load(f))


[1 2]
[1 3]


EOFError: No data left in file

In [139]:
a

array([1, 2])

In [140]:
a = np.load('test.npy')

In [141]:
a

array([1, 2])